In [1]:
from datasets import load_dataset 

dataset = load_dataset("../data/images/", split="train")

/home/ajj/anaconda3/envs/t1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from torch.utils.data import Dataset, DataLoader

MAX_PATCHES = 1024

class ImageCaptioningDataset(Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        encoding = self.processor(images=item["image"], return_tensors="pt", add_special_tokens=True, max_patches=MAX_PATCHES)
        
        encoding = {k:v.squeeze() for k,v in encoding.items()}
        encoding["text"] = item["text"]
        return encoding

In [7]:
from transformers import Pix2StructForConditionalGeneration, Pix2StructProcessor
from PIL import Image

model = Pix2StructForConditionalGeneration.from_pretrained("google/pix2struct-infographics-vqa-large").to("cuda")
processor = Pix2StructProcessor.from_pretrained("google/pix2struct-infographics-vqa-large")

In [21]:
def collator(batch):
  new_batch = {"flattened_patches":[], "attention_mask":[]}
  texts = [item["text"] for item in batch]
  
  text_inputs = processor(text=texts, padding="max_length", return_tensors="pt", add_special_tokens=True, max_length=20)
  
  new_batch["labels"] = text_inputs.input_ids
  
  for item in batch:
    new_batch["flattened_patches"].append(item["flattened_patches"])
    new_batch["attention_mask"].append(item["attention_mask"])
  
  new_batch["flattened_patches"] = torch.stack(new_batch["flattened_patches"])
  new_batch["attention_mask"] = torch.stack(new_batch["attention_mask"])

  return new_batch
     

In [9]:
from datasets import load_dataset 

dataset = load_dataset("json", data_files="../data/qas/infographicsVQA_train_v1.0.json", split="train")

Generating train split: 1 examples [00:00,  4.80 examples/s]
Generating validation split: 0 examples [00:00, ? examples/s]


DatasetGenerationError: An error occurred while generating the dataset

In [22]:
train_dataset = ImageCaptioningDataset(dataset, processor)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=2, collate_fn=collator)

In [3]:
import json
import os
from PIL import Image
import torch
from torch.utils.data import Dataset
from torchvision import transforms

class InfographicsVQADataset(Dataset):
    def __init__(self, json_file='data/qas/your_json_file.json', img_dir='data/images', transform=None):
        """
        Args:
            json_file (string): JSON 파일의 경로.
            img_dir (string): 이미지 파일이 저장된 폴더의 경로.
            transform (callable, optional): 샘플에 적용될 Optional transform.
        """
        with open(json_file, 'r') as f:
            self.vqa_data = json.load(f)['data']
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.vqa_data)

    def __getitem__(self, idx):
        vqa_example = self.vqa_data[idx]
        img_path = os.path.join(self.img_dir, vqa_example['image_local_name'])
        image = Image.open(img_path).convert('RGB')  # 이미지를 RGB로 변환
        
        # 이미지 전처리 적용
        if self.transform:
            image = self.transform(image)
        
        # 다른 정보들도 필요에 따라 반환할 수 있습니다.
        sample = {'image': image, 
                  'question': vqa_example['question'], 
                  'answers': vqa_example['answers']}

        return sample

# 이미지 전처리 파이프라인 정의
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# 데이터셋 인스턴스 생성
infographics_vqa_dataset = InfographicsVQADataset(json_file='../data/qas/infographicsVQA_train_v1.0.json', img_dir='../data/images', transform=transform)

# DataLoader를 통해 배치 처리 준비
from torch.utils.data import DataLoader

dataloader = DataLoader(infographics_vqa_dataset, batch_size=4, shuffle=True)


In [4]:
dataloader.dataset[0]

{'image': tensor([[[0.3490, 0.3373, 0.3333,  ..., 0.3373, 0.3373, 0.3373],
          [0.3490, 0.3373, 0.3333,  ..., 0.3373, 0.3373, 0.3373],
          [0.3490, 0.3373, 0.3333,  ..., 0.3373, 0.3373, 0.3373],
          ...,
          [0.3490, 0.3373, 0.3333,  ..., 0.3373, 0.3373, 0.3373],
          [0.3490, 0.3373, 0.3333,  ..., 0.3373, 0.3373, 0.3373],
          [0.3490, 0.3373, 0.3333,  ..., 0.3373, 0.3373, 0.3373]],
 
         [[0.3137, 0.3020, 0.2980,  ..., 0.3020, 0.3020, 0.3020],
          [0.3137, 0.3020, 0.2980,  ..., 0.3020, 0.3020, 0.3020],
          [0.3137, 0.3020, 0.2980,  ..., 0.3020, 0.3020, 0.3020],
          ...,
          [0.3137, 0.3020, 0.2980,  ..., 0.3020, 0.3020, 0.3020],
          [0.3137, 0.3020, 0.2980,  ..., 0.3020, 0.3020, 0.3020],
          [0.3137, 0.3020, 0.2980,  ..., 0.3020, 0.3020, 0.3020]],
 
         [[0.2941, 0.2824, 0.2784,  ..., 0.2824, 0.2824, 0.2824],
          [0.2941, 0.2824, 0.2784,  ..., 0.2824, 0.2824, 0.2824],
          [0.2941, 0.2824, 0.27

In [5]:
model

NameError: name 'model' is not defined

In [8]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import AdamW, get_linear_schedule_with_warmup

# 모델과 토크나이저 로드
model_name = "google/pix2struct-infographics-vqa-large"  # 예시 모델명; 실제 모델명으로 대체해야 함
model = Pix2StructForConditionalGeneration.from_pretrained(model_name).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 모든 파라미터를 기본적으로 고정
for param in model.parameters():
    param.requires_grad = False

# 인코더와 디코더의 마지막 N개 레이어를 학습 가능하게 설정
N = 2  # 예를 들어, 마지막 2개의 레이어를 학습 가능하게 설정합니다.

# 인코더 레이어
for layer in model.encoder.encoder.layer[-N:]:
    for param in layer.parameters():
        param.requires_grad = True

# 디코더 레이어
for layer in model.decoder.layer[-N:]:
    for param in layer.parameters():
        param.requires_grad = True

# 옵티마이저와 스케줄러 설정
optimizable_params = [p for p in model.parameters() if p.requires_grad]
optimizer = AdamW(optimizable_params, lr=5e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=1000) # 예시 값

# 학습 루프 예시
model.train()
for epoch in range(1):  # 실제 사용 시, 적절한 에폭 수 설정 필요
    for batch in dataloader:
        questions = batch['question']  # 'question' 키를 사용해 질문 데이터에 접근
        
        # Tokenizer를 사용하여 텍스트 데이터를 모델이 처리할 수 있는 형태로 변환
        inputs = tokenizer(questions, return_tensors="pt", padding=True, truncation=True, max_length=512).to("cuda")
        
        # 주의: 이 예제에서는 정답 데이터('answers')를 어떻게 처리해야 할지 명시하지 않았습니다.
        # VQA 작업의 경우, 정답은 모델이 예측해야 할 출력으로 사용되거나, 모델 학습에 다르게 활용될 수 있습니다.
        
        # 모델 입력 준비
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']
        
        # 모델 실행 및 손실 계산 (손실 계산 방법은 작업에 따라 다름)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)  # labels 부분은 작업에 따라 조정 필요
        loss = outputs.loss
        
        # 역전파
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

# for epoch in range(1):  # 실제 사용 시, 적절한 에폭 수 설정 필요
    # 데이터 로더에서 배치 가져오기 (가정)
    for batch in dataloader:
        inputs = tokenizer(batch['input_text'], return_tensors="pt", padding=True, truncation=True, max_length=512)
        outputs = tokenizer(batch['target_text'], return_tensors="pt", padding=True, truncation=True, max_length=512)
        
        # 모델 입력 준비
        input_ids = inputs.input_ids
        attention_mask = inputs.attention_mask
        labels = outputs.input_ids
        
        # 모델 실행 및 손실 계산
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        
        # 역전파
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

# 학습 후 모델 저장
model.save_pretrained("./finetuned_model")
tokenizer.save_pretrained("./finetuned_model")


/home/ajj/anaconda3/envs/t1/lib/python3.10/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


KeyError: 'input_text'

In [3]:
from torch.utils.data import Dataset, DataLoader
import os
import pandas as pd
from PIL import Image
import torchvision.transforms as transforms

class VQADataset(Dataset):
    def __init__(self, annotations_file, img_dir, tokenizer, max_length=512):
        """
        annotations_file: 답변과 질문이 포함된 CSV 파일의 경로
        img_dir: 이미지 파일이 저장된 디렉토리의 경로
        tokenizer: 텍스트 데이터를 토크나이징하기 위한 tokenizer
        max_length: 토큰화된 입력의 최대 길이
        """
        self.img_dir = img_dir
        self.df = pd.read_csv(annotations_file)
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
        ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.img_dir, self.df.iloc[idx, 0])
        image = Image.open(img_name)
        image = self.transform(image)
        question = self.df.iloc[idx, 1]
        answer = self.df.iloc[idx, 2]

        # 여기서는 간단히 입력 텍스트를 질문과 결합합니다.
        input_text = "[QUESTION] " + question + " [ANSWER] " + answer
        inputs = self.tokenizer(input_text, return_tensors="pt", max_length=self.max_length, padding="max_length", truncation=True)

        return {
            'input_ids': inputs['input_ids'].squeeze(0),  # 배치 차원 제거
            'attention_mask': inputs['attention_mask'].squeeze(0),  # 배치 차원 제거
            'images': image
        }


In [4]:
# Dataset 인스턴스 생성
annotations_file = "../data/qasinfographicsVQA_train_v1.0.csv"
img_dir = "../data/images"
dataset = VQADataset(annotations_file, img_dir, tokenizer, max_length=512)

# DataLoader 설정
batch_size = 4
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


NameError: name 'tokenizer' is not defined